In [ ]:
# All configs

base_path = "../Alfrodull/experiments/ubelix/"

thor_output_dir = "walf_spinup"
planet_name = "Wasp43b"

FIGSIZE = (10, 7)

In [ ]:
import mjolnir_plot_helper as mph

import pathlib

#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

import re

import h5py

from file_helpers import (get_path_matching_regex_with_groups,
                          get_path_matching_regex)

import hamarr as ham

import IPython.display as disp

from PIL import Image, ImageDraw, ImageFont, ImageOps
 
import imageio

from pygifsicle import optimize

In [ ]:
exp_path = pathlib.Path(base_path) / thor_output_dir

In [ ]:
# get count of output files

# get output files
outputs = get_path_matching_regex_with_groups(exp_path, f"esp_output_{planet_name}_(\d+).h5" )

# sort'em
d = {}

for o in outputs:
    p = o['path']
    (idx_chr,) = o['groups']
    idx = int(idx_chr)
    # print(f"{idx} - {p}")
    d[idx] = p
    
sorted_files = sorted(d.items())


first_idx = sorted_files[0][0]
last_idx = sorted_files[-2][0] 

In [ ]:
class args:
    pass

file_idx = 1

args.pview = ['fuptot']
args.file = [str(exp_path)]
args.simulation_ID = ['auto']
args.initial_file = [file_idx]
args.last_file = [file_idx]
args.horizontal_lev = [2.5e2]
args.vertical_top = ['default']
args.split_layer = ['no_split']
args.coordinate_sys = ['icoh']
args.lmax_adjust = [0]
args.slice = [0, 180]
args.maketable = False
args.no_pressure_log = False
args.latlonswap = False
args.vcoord = ['pressure']
args.pgrid_ref = [f'pgrid_{file_idx}_{file_idx}_1.txt']
args.clevels = [40]

In [ ]:
figures_dir = exp_path / "figures"
if not figures_dir.exists():
    figures_dir.mkdir(parents=True, exist_ok=True)

In [ ]:


def plot_anim_steps(first_idx, last_idx, args, plot_filename, overwrite_anim=False):
    stride = 1
    overwrite = False
    output_image = exp_path / "figures" / plot_filename
    
    if output_image.exists() and not overwrite_anim:
        print(f"{output_image} already exists, skipping. Set overwrite_anim = True to force")
        return output_image
    
    fps = 10
    writer = imageio.get_writer(str(output_image), fps=fps)


    # for dev, force to use a small number of files
    #last_idx  = 20
    fig, ax = plt.subplots(1,1, figsize=FIGSIZE)    

    for i in range(first_idx, last_idx):
    
        print(f"plotting: {i: 5}/{last_idx}\r", end='')
        args.initial_file = [i]
        args.last_file = [i]
        args.pgrid_ref = [f'pgrid_{i}_{i}_1.txt']

    
    
        fig.clear()
        ax = fig.add_subplot(111)
    
        pgrid_ref = exp_path / args.pgrid_ref[0]
        if pgrid_ref.exists():
            pass
        else:
            ham.define_Pgrid(args.file[0],
                             planet_name,
                             args.initial_file[0],
                             args.last_file[0],
                             stride,
                             overwrite=overwrite)
    
        mph.make_plot(args,False, axis=(fig, ax))
    
    
        canvas = fig.canvas

        # Option 2: Retrieve a view on the renderer buffer...
        canvas.draw()
        buf = canvas.buffer_rgba()
        # ... convert to a NumPy array ...
        X = np.asarray(buf)
        # ... and pass it to PIL.
        #im = Image.fromarray(X)
    
        writer.append_data(X)
    
    
    writer.close()

    optimize(str(output_image))
    
    return output_image

def plot_steps(plot_type, first_idx, last_idx, args):
    fig, ((ax_first, ax_last),
          (ax_second, ax_mid)) = plt.subplots(2,2, figsize=FIGSIZE)

    i = first_idx
    args.pview = [plot_type]
    args.initial_file = [i]
    args.last_file = [i]
    args.pgrid_ref = [f'pgrid_{i}_{i}_1.txt']


    mph.make_plot(args,False, axis=(fig, ax_first))
    ttl = ax_first.get_title()
    ax_first.set_title(ttl + "\n" + "First (idx=0)")

    i = last_idx
    args.pview = [plot_type]
    args.initial_file = [i]
    args.last_file = [i]
    args.pgrid_ref = [f'pgrid_{i}_{i}_1.txt']


    mph.make_plot(args,False, axis=(fig, ax_last))
    ttl = ax_last.get_title()
    ax_last.set_title(ttl + "\n" + f"Last (idx={i})")

    i = 1
    args.pview = [plot_type]
    args.initial_file = [i]
    args.last_file = [i]
    args.pgrid_ref = [f'pgrid_{i}_{i}_1.txt']


    mph.make_plot(args,False, axis=(fig, ax_second))
    ttl = ax_second.get_title()
    ax_second.set_title(ttl + "\n" + "First step (idx=1)")


    i = (last_idx - first_idx) // 2
    args.pview = [plot_type]
    args.initial_file = [i]
    args.last_file = [i]
    args.pgrid_ref = [f'pgrid_{i}_{i}_1.txt']


    mph.make_plot(args,False, axis=(fig, ax_mid))
    ttl = ax_mid.get_title()
    ax_mid.set_title(ttl + "\n" + f"Middle step (idx={i})")


In [ ]:
def plot_anim(plotname, plotfile, first, last, args):
    print(f"plotting {plotname} to {plotfile}")
    args.pview = [plotname]
    output_image = plot_anim_steps(first_idx, last_idx, args, plotfile)
    im = Image.open(output_image)
    return disp.Image(output_image, format="gif")

In [ ]:
# plots = [         
#          ("TP", "TP_anim.gif"),
#          ("Tver", "Tver_anim.gif"),
#          ("Tlonver", "Tver_anim.gif"),
#          ("fuptot", "fuptot_anim.gif"),
#          ("fdowntot", "fdowntot_anim.gif"),
#          ("fnet", "fnet_anim.gif"),
#          ("qheat", "qheat_anim.gif"),
#          ("futprof", "futprof_anim.gif"),
#          ("fdtprof", "fdtprof_anim.gif"),
#          ("fnetprof", "fnetprof_anim.gif"),
#         ]

# for plotname, plotfile in plots:
#     print(f"plotting {plotname} to {plotfile}")
#     args.pview = [plotname]
#     output_image = plot_anim_steps(first_idx, last_idx, args, plotfile)
#     im = Image.open(output_image)
#     disp.Image(output_image, format="gif")



# TP Profile

In [ ]:
plot_anim( "TP", "TP_anim.gif", first_idx, last_idx, args)

In [ ]:
plot_steps("TP", first_idx, last_idx, args)

# Tver

In [ ]:
plot_anim( "Tver", "Tver_anim.gif", first_idx, last_idx, args)

In [ ]:
plot_steps("Tver", first_idx, last_idx, args)

# Tlonver

In [ ]:
plot_anim( "Tlonver", "Tlonver_anim.gif", first_idx, last_idx, args)

In [ ]:
plot_steps("Tlonver", first_idx, last_idx, args)

# Qheat

In [ ]:
plot_anim("qheat", "qheat_anim.gif", first_idx, last_idx, args)    

In [ ]:
plot_steps("qheat", first_idx, last_idx, args)

# Total Upward Flux 

## Profile

In [ ]:
plot_anim("futprof", "futprof_anim.gif", first_idx, last_idx, args)    

In [ ]:
plot_steps("futprof", first_idx, last_idx, args)

## Horizontal

In [ ]:
plot_anim( "fuptot", "fuptot_anim.gif", first_idx, last_idx, args)

In [ ]:
plot_steps("fuptot", first_idx, last_idx, args)

# Total Downward Flux 

## Profile

In [ ]:
plot_anim("fdtprof", "fdtprof_anim.gif", first_idx, last_idx, args)    

In [ ]:
plot_steps("fdtprof", first_idx, last_idx, args)

## Horizontal

In [ ]:
plot_anim("fdowntot", "fdowntot_anim.gif", first_idx, last_idx, args)

In [ ]:
plot_steps("fdowntot", first_idx, last_idx, args)

# Total Net Flux 

## Profile

In [ ]:
plot_anim("fnetprof", "fnetprof_anim.gif", first_idx, last_idx, args)

In [ ]:
plot_steps("fnetprof", first_idx, last_idx, args)

## Horizontal

In [ ]:
plot_anim("fnet", "fnet_anim.gif", first_idx, last_idx, args)

In [ ]:
plot_steps("fnet", first_idx, last_idx, args)